In [ ]:
!pip install boto3==1.35.55 botocore==1.35.55

In [ ]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

if not all([aws_access_key_id, aws_secret_access_key, endpoint_url, region_name, bucket_name]):
    raise ValueError("One or more connection variables are empty.  "
                     "Please check your connection to an S3 bucket.")

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    next_version = get_next_version(list_objects("models"))
    
    num_files = 0
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            local_file_path = os.path.join(root, filename)
            file_path = os.path.join('distance', next_version, filename)
            s3_key = os.path.join(s3_prefix, file_path)
            print(f"{local_file_path} -> {s3_key}")
            #bucket.upload_file(local_file_path, s3_key)
            num_files += 1
    return num_files


def list_objects(prefix):
    objects = []
    filter = bucket.objects.filter(Prefix=prefix)
    for obj in filter.all():
        objects.append(obj.key)
    return objects


def get_next_version(models):
    model_versions = [int(x.split('models/distance/')[1].split('/')[0]) for x in models]
    next_version = str(max(model_versions) + 1)
    return next_version

In [ ]:
local_models_directory = "models"

if not os.path.isdir(local_models_directory):
    raise ValueError(f"The directory '{local_models_directory}' does not exist.  "
                     "Did you finish training the model in the previous notebook?")

num_files = upload_directory_to_s3("models", "models")

if num_files == 0:
    raise ValueError("No files uploaded.  Did you finish training and "
                     "saving the model to the \"models\" directory?  "
                     "Check for \"models/fraud/1/model.onnx\"")


In [ ]:
get_next_version(list_objects("models"))